# AB Comb Polymers in Potential Fields

This tutorial demonstrates how to compute partition functions, propagators, and concentrations of **comb polymers** (backbone with side chains) in external potential fields.

**What you'll learn:**
- Setting up comb polymer architectures with backbone and side chains
- Computing partition functions and concentrations

**Prerequisites:** Complete `00_QuickStart.ipynb` and `Diblock.ipynb` first.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded OpenMP
os.environ["MKL_NUM_THREADS"] = "1"      # Single-threaded MKL

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from polymerfts import PropagatorSolver
import polymerfts

## 2. Simulation Parameters

| Parameter | Value | Description |
|-----------|-------|-------------|
| $N$ | 100 | Reference chain length (number of statistical segments) |
| $\Delta s$ | 0.01 | Contour step size (= $1/N$) |
| $L_x, L_y$ | $5.0 R_0$ | Box dimensions in units of $R_0 = b\sqrt{N}$ |
| $m_x, m_y$ | 64 | Grid points per dimension |
| $b_A/b, b_B/b$ | 1.0 | Statistical segment lengths (relative to reference $b$) |
| Backbone | A-type | Two A blocks of length 0.5 each |
| Side chains | B-type | Two B blocks of length 0.5 each |

In [ ]:
# Simulation parameters
nx = [64, 64]              # Grid points
lx = [5.0, 5.0]            # Box size (in units of R0 = b*sqrt(N))
ds = 0.01                  # Contour step interval

## 3. Create the Solver

We use `PropagatorSolver` to set up a comb polymer with an A-type backbone and B-type side chains.

Graph notation:
```
  B(3)         B(4)
   |            |
  A(0)---A(1)---A(2)
```
- Backbone: nodes 0-1-2 connected by A blocks
- Side chains: B blocks at nodes 0 and 1

In [ ]:
# Create solver
solver = PropagatorSolver(
    nx=nx,
    lx=lx,
    ds=ds,
    bond_lengths={"A": 1.0, "B": 1.0},
    bc=["periodic", "periodic", "periodic", "periodic"],
    chain_model="continuous",
    method="pseudospectral",
    platform="cpu-mkl",
    reduce_memory_usage=False
)

# Add comb polymer: A backbone with B side chains
solver.add_polymer(
    volume_fraction=1.0,
    blocks=[
        ["A", 0.5, 0, 1],  # Backbone segment 1
        ["A", 0.5, 1, 2],  # Backbone segment 2
        ["B", 0.5, 0, 3],  # Side chain at node 0
        ["B", 0.5, 1, 4],  # Side chain at node 1
    ]
)

print(solver.info)

## 4. Set Potential Fields

We create sinusoidal potential fields that are opposite for A and B monomers.

In [ ]:
# Create sinusoidal potential fields (opposite for A and B)
w_A = np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))
w_B = -w_A  # Opposite preference

# Visualize the potential fields
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Potential Fields", fontsize=14)

im0 = axes[0].imshow(w_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis')
im1 = axes[1].imshow(w_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis')
axes[0].set(title='$w_A$ (A monomers)', xlabel='y', ylabel='x')
axes[1].set(title='$w_B$ (B monomers)', xlabel='y', ylabel='x')

plt.colorbar(im0, ax=axes[0])
plt.colorbar(im1, ax=axes[1])
plt.tight_layout()
plt.show()

## 5. Compute Partition Function

The propagators for A-type blocks, $(v,u) \in \{(0,1),(1,2),(1,0),(2,1)\}$:

$$\frac{\partial q^{v \rightarrow u}_0(\mathbf{r}, s)}{\partial s} = \frac{b_A^2}{6}\nabla^2 q^{v \rightarrow u}_0(\mathbf{r}, s) - w_A(\mathbf{r}) q^{v \rightarrow u}_0(\mathbf{r}, s), \quad s\in [0, f_A]$$

The propagators for B-type blocks in forward direction, $(v,u) \in \{(3,0),(4,1)\}$:

$$\frac{\partial q^{v \rightarrow u}_0(\mathbf{r}, s)}{\partial s} = \frac{b_B^2}{6}\nabla^2 q^{v \rightarrow u}_0(\mathbf{r}, s) - w_B(\mathbf{r}) q^{v \rightarrow u}_0(\mathbf{r}, s), \quad s\in [0, f_B]$$

The propagators for B-type blocks in backward direction with aggregation:

$$\frac{\partial}{\partial s}\left[q^{0 \rightarrow 3}_0(\mathbf{r}, s)+q^{1 \rightarrow 4}_0(\mathbf{r}, s) \right] = \frac{b_B^2}{6}\nabla^2 \left[q^{0 \rightarrow 3}_0(\mathbf{r}, s)+q^{1 \rightarrow 4}_0(\mathbf{r}, s) \right] - w_B(\mathbf{r}) \left[q^{0 \rightarrow 3}_0(\mathbf{r}, s)+q^{1 \rightarrow 4}_0(\mathbf{r}, s) \right], \quad s\in [0, f_B]$$

Single chain partition function:

$$Q_0 = \frac{1}{V}\int d\mathbf{r}\, q^{0 \rightarrow 1}_0(\mathbf{r}, s) \cdot q^{1 \rightarrow 0}_0(\mathbf{r}, f_A-s)$$

In [ ]:
# Compute propagators
solver.compute_propagators({"A": w_A.flatten(), "B": w_B.flatten()})

# Get single chain partition function
Q = solver.get_partition_function(polymer=0)
print(f"Single chain partition function Q = {Q:.6f}")
print(f"\nInterpretation: Q > 1 means the potential field favors certain conformations")

## 6. Compute Concentrations

$$\phi_A(\mathbf{r}) = \frac{1}{Q_0} \sum_{(v,u)\in\{(0,1),(1,2)\}} \int_{0}^{f_A} ds\, q^{v \rightarrow u}_0(\mathbf{r}, s) \cdot q^{u \rightarrow v}_0(\mathbf{r}, f_A-s)$$

$$\phi_B(\mathbf{r}) = \frac{1}{Q_0} \sum_{(v,u)\in\{(0,3),(1,4)\}} \int_{0}^{f_B} ds\, q^{v \rightarrow u}_0(\mathbf{r}, s) \cdot q^{u \rightarrow v}_0(\mathbf{r}, f_B-s) = \frac{1}{Q_0} \int_{0}^{f_B} ds\, \left[q^{0 \rightarrow 3}_0(\mathbf{r}, s)+q^{1 \rightarrow 4}_0(\mathbf{r}, s)\right] \cdot q^{3 \rightarrow 0}_0(\mathbf{r}, f_B-s)$$

In [ ]:
# Compute concentrations
solver.compute_concentrations()

# Get ensemble-averaged concentrations
phi_A = np.reshape(solver.get_concentration("A"), nx)
phi_B = np.reshape(solver.get_concentration("B"), nx)

# Visualize concentrations
vmin = min(phi_A.min(), phi_B.min())
vmax = max(phi_A.max(), phi_B.max())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Monomer Concentrations", fontsize=14)

im0 = axes[0].imshow(phi_A, extent=(0, lx[1], 0, lx[0]), origin='lower', 
                      cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0].set(title='$\phi_A$ (A concentration)', xlabel='y', ylabel='x')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(phi_B, extent=(0, lx[1], 0, lx[0]), origin='lower', 
                      cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[1].set(title='$\phi_B$ (B concentration)', xlabel='y', ylabel='x')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

print("\nObservation: A monomers (backbone) concentrate where w_A < 0")
print("             B monomers (side chains) concentrate where w_B < 0")

## 7. Summary

In this tutorial, we learned:

1. **Comb polymer architecture**: Backbone with side chains attached at branching points
2. **Graph notation**: Nodes define branching points, blocks connect adjacent nodes
3. **PropagatorSolver** handles the setup automatically

### Next Steps

- **Multi-arm stars**: See `BranchedMultiArmStar.ipynb` for star polymer architectures
- **Polymer mixtures**: See `Mixture.ipynb`
- **Accessing individual propagators**: Set `reduce_memory_usage=False` to store all propagator steps (see `Diblock.ipynb` for examples)